In [ ]:
#In this program, I got all the information related to process Regimen Especial. 
#I can get information for semester. Example: Start Date:'2017-07-01', End Date:'2017-12-31'

import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import lxml.html as lh
import os
from datetime import datetime
import dateparser


def get_table_with_links(driver, startdate, enddate):
   
    url = 'https://www.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/buscarProcesoRE.cpe?op=R'
    idTipoContratacion = '386'
    hospital = "ABEL GILBERT PONTON"

    driver.get(url)  # Abrir una nueva instancia del browser
    driver.maximize_window()
    x=0
    # storing the current window handle to get back to dashbord 
    mainPage = driver.current_window_handle 
    
    #removing the alert displayed in the black box
    driver.find_element_by_xpath("//div[@id='alertInicial']//div[2]/button").click() 

    
    #click en Buscar Entidad  
    driver.find_element_by_xpath("//table//table//table//tbody//tbody//a").click()
    #busqueda de entidades
    # changing the handles to access login page 
    for handle in driver.window_handles: 
        if handle != mainPage: 
            searchgPage = handle 
    
    # change the control to signin page         
    driver.switch_to.window(searchgPage) 
    
    txtEmpresa = driver.find_element_by_xpath("//input[@id='txtEmpresa']")
    txtEmpresa.send_keys(hospital) 
    driver.find_element_by_xpath("//img[@id='btnBuscar']").click()
    
    onclickValue = driver.find_element_by_xpath("//body/form[@id='frmDatos']/table/tbody/tr/td/a[1]").get_attribute("onclick")
    #print(onclickValue)

    entidadContratante = onclickValue.replace('javascript:SetSelectedItem', '').replace('(', '').replace(')', '')
    splitUp = entidadContratante.split(",")
    
    driver.switch_to.window(mainPage)
    driver.execute_script('document.getElementById("txtEntidadContratante").removeAttribute("readonly")')
    nombreEntidad = driver.find_element_by_xpath("//textarea[@id='txtEntidadContratante']") 
    nombreEntidad.send_keys(splitUp[1].replace("'",""))

    #asingar el codigo de la entidad al input(hidden)
    driver.execute_script('document.getElementById("cmbEntidad").setAttribute("value",' + splitUp[0] + ')'  )
    
    #Seleccionar Tipo de Contratación
    #selectTipoContratacion = Select(driver.find_element_by_id("txtCodigoTipoCompra")) 
    #selectTipoContratacion.select_by_value(idTipoContratacion)
    
    datepicker = driver.find_element_by_xpath("//img[@name='ico_f_inicio']")
    datepicker.click()
    fechaInicio= driver.find_element_by_xpath("//input[@id='f_inicio']")
    driver.execute_script('document.getElementById("f_inicio").removeAttribute("readonly")')
    fechaInicio.clear()
    fechaInicio.send_keys(startdate)


    datepicker2 = driver.find_element_by_xpath("//img[@name='ico_f_fin']")
    datepicker2.click()
    fechaFin= driver.find_element_by_xpath("//input[@id='f_fin']")
    driver.execute_script('document.getElementById("f_fin").removeAttribute("readonly")')
    fechaFin.clear()
    fechaFin.send_keys(enddate)

    #click en Buscar
    driver.find_element_by_xpath("//body/div[@id='contenedor']/div/div/form[@id='frmDatos']/table/tbody/tr/td/table/tbody/tr/td/table[@id='toolbarbotones']/tbody/tr/td[1]/a[1]/img[1]").click()
    
    readTableResults(driver, startdate, enddate)
    
    

def readTableResults(driver, startdate, enddate):

    totalRecords = driver.find_element_by_xpath("//table[2]//tbody[1]//tr[1]//td[1]").text
    totalRecords = totalRecords.split(" ")
    #numberPages = int(int(totalRecords[6])/20) + 1
    
    if int(totalRecords[6]) <= 20:
        numberPages = 1
    else:
        numberPages = int(int(totalRecords[6])/20) + 1

    print (numberPages)
    parentDir = "D:\hagp\Data"
    
    df = pd.DataFrame()
    # Create the pandas DataFrame 
    treatLinks = []
    treatCodeProcess = []
    treatStatus = []
    treatExpenses = []
    treatDate = []

    for j in range(0,numberPages,1):
        
        treatLinks1 = []
        treatCodeProcess1 = []
        for i in driver.find_elements_by_xpath("//div[@id='divProcesos']//table[1]//tbody[1]//tr//td[1]//a"):
            try:
                link = i.get_attribute('href')
                treatLinks1.append(link)
                treatCodeProcess1.append(i.text)

            except:
                treatLinks1.append('')
                treatCodeProcess1.append('')
        treatLinks.extend(treatLinks1)
        treatCodeProcess.extend(treatCodeProcess1)

        treatStatus1 = []
        elements = driver.find_elements_by_xpath("//div[@id='divProcesos']//table[1]//tbody[1]//tr//td[4]")
        for element in elements:
            try:
                treatStatus1.append(element.text)
        
            except:
                treatStatus1.append('')

        treatStatus1.pop(0)
        treatStatus.extend(treatStatus1)
        
        #Expenses without taxes
        treatExpenses1 = []
        elements =  driver.find_elements_by_xpath("//div[@id='divProcesos']//table[1]//tbody[1]//tr//td[6]")
        for element in elements:
            try:
                treatExpenses1.append(str(element.text).replace("$","").replace(",",""))
                    
            except:
                treatExpenses1.append('')
        
        treatExpenses1.pop(0) 
        treatExpenses.extend(treatExpenses1)
        

        #Date posted
        treatDate1 = []
        elements = driver.find_elements_by_xpath("//div[@id='divProcesos']//table[1]//tbody[1]//tr//td[7]")
        for element in elements :
            try:
                treatDate1.append(element.text)
                    
            except:
                treatDate1.append('')
                    
        treatDate1.pop(0) 
        treatDate.extend(treatDate1) 
        

        if j+1<numberPages: driver.find_element_by_xpath("//div[@id='divProcesos']//td[3]//a[1]").click()
        j+=1    
    
    df['Code'] = treatCodeProcess
    df['Link'] = treatLinks
    df['Status'] = treatStatus
    df['Expenses'] = treatExpenses
    df['Date'] = treatDate

    driver.quit()
  
    dp = dateparser.parse(startdate)
    #startMonth = datetime.datetime.strptime(startdate, '%d/%m/%Y')
    year = dp.year
    path = os.path.join(parentDir, str(year))
    try: 
        os.makedirs(path, exist_ok = True) 
        print("Directory '%s' created successfully" % str(year)) 
    except OSError as error: 
        print("Directory '%s' can not be created" % str(year)) 

    newFile = path + getNameFile(enddate)
    df.to_csv(newFile, encoding='utf-8-sig')

def getNameFile(enddate):
    
    nameFile = ""
    months = [1,2,3,4,5,6]
    dp = dateparser.parse(enddate)
    print(dp)
    #dateObj = datetime.datetime.strptime(startdate.replace("-","/"), '%d/%m/%Y')
    month = dp.month
    if month in months:
        semester = 1
    else:
        semester = 2

    print (semester)
    nameFile = "\RE" + "-" + str(dp.year) + "-" + str(semester) + ".csv"

    return nameFile

options = Options()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument("--headless")
my_driver = webdriver.Chrome(options=options,executable_path=r'D:\webdrivers\chromedriver.exe')
 
get_table_with_links(my_driver, '2017-07-01', '2017-12-31')
